In [ ]:
#Import modules and declare TIC

%matplotlib widget

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import lightkurve as lk
from scipy.interpolate import make_interp_spline as spline
import scipy.signal as signal
import matplotlib
import plotly.express as px
from astropy.time.core import Time

plt.style.use('seaborn-v0_8-darkgrid')

TIC = 'TIC 219107776'

In [ ]:
#Download the data and stitch it together

search_results = lk.search_lightcurve(TIC, cadence='short', author='SPOC')

#Enter the data index to be used                                                                                        [INPUT NEEDED]
use_till = 5

lcs = []
for i in range(use_till):
    try:
        lcs.append(search_results[i].download(quality_bitmask='hardest'))
    except:
        pass

search_results


In [ ]:
flux = np.array(lcs[0].flux, dtype=float)
time = np.array(lcs[0].time.jd)

flux = np.append(flux, np.array(lcs[1].flux, dtype=float))
time = np.append(time, np.array(lcs[1].time.jd))

cadence_in_days = ((np.median(np.diff(time[:100])) * 86400).round())/86400

In [ ]:
plt.figure(figsize = (15, 5))
plt.plot(time, flux, 'k.', lw = 0.5)
plt.plot(time, signal.savgol_filter(flux, 1000, 4), 'r', lw = 0.5)
plt.plot(time_smooth, flux_smooth, 'b', lw = 0.5)
plt.plot(peaktimes, range(len(peaktimes)), 'x')
plt.show()

In [ ]:
time_smooth = np.linspace(time.min(), time.max(), len(time) * 8)
flux_smooth = spline(time, flux, k = 3)(time_smooth)

In [ ]:
plt.figure()
plt.plot(np.diff(time))

peaks, p = signal.find_peaks(np.diff(time), height = cadence_in_days * 10)
peaks
plt.plot(peaks, p['peak_heights'], 'x')
p['peak_heights']
plt.show()
peaktimes = time[peaks]


In [ ]:
print(len(peaks), len(p['peak_heights']))

In [ ]:
print(len(peaktimes))

In [ ]:
for i in peaks:
    t = np.array(time[i], time[i+1])
    f = np.array(flux[i], flux[i+1])
    df = pd.DataFrame({'time': t, 'flux': f})
    df.set_index('time', inplace = True)
    df = df.reindex(np.arange(time[i], time[i+1], cadence_in_days))
    df.interpolate(inplace = True)
    df.reset_index(inplace = True)

In [ ]:
px.line(x = time, y = signal.savgol_filter(flux, 101, 3)).show()